In [3]:
import numpy as np
import pandas as pd
import os

In [2]:
from sklearn.metrics import matthews_corrcoef
# usage: matthews_corrcoef(y_true, y_pred)

In [3]:
def write_mcc_to_file(results_list, filename):    
    df_to_write = pd.DataFrame(data=results_list, \
                               index=['Whole', 'Gender_Male', 'Gender_Female',\
                                     'Ethnicity_White', 'Ethnicity_Black',\
                                     'Ethnicity_Hispanic', 'Ethnicity_Asian',\
                                     'Age<30', '30<=Age<40', '40<=Age<50',\
                                     '50<=Age<60', '60<=Age<70',\
                                     '70<=Age<80', '80<=Age<90', 'Age>=90'],\
                              columns=['MCC'])
    #print(df_to_write)
    df_to_write.to_csv(filename)

In [4]:
def evaluate_all_subgroup_all_threshold(df_results, cancer, folder_name, valid_or_test):
    """
    generate mcc for all threshold, containing all subgroup results
    for one file (valid or test, for a single epoch/model)
    
    true_y: the label
    score_y: predicted score/probability
    folder_name: directory of the epoch/model, result file will be written to this dir
    """
    if cancer =='breast':
        # flip bcs classes for mlp models
        df_results['true y'] = (df_results['true y'] == 0).astype(np.int)
        df_results['score y'] = 1 - df_results['score y']

        
    """
    for whole group table
    takes a data frame contains all results
    returns a list with subgroups analysis
    """
    # divde into groups
    # sex groups, 0 - female, 1 - male
    male = df_results[df_results['Sex 1'] == 1]
    female = df_results[df_results['Sex 1'] == 0]
    # race groups
    # recode 1 - white, 2 - black, 4 - asian, origin recode 1 - hispanic
    white = df_results[df_results['Race recode Y 1'] == 1]
    black = df_results[df_results['Race recode Y 2'] == 1]
    hispanic = df_results[df_results['Origin Recode NHIA 1'] == 1]
    asian = df_results[df_results['Race recode Y 4'] == 1]
    # age groups
    _30_and_below = df_results[df_results['Age at diagnosis continuous'] < 30]
    _31_to_40 = df_results[(df_results['Age at diagnosis continuous'] >= 30) & \
                                   (df_results['Age at diagnosis continuous'] < 40)]
    _41_to_50 = df_results[(df_results['Age at diagnosis continuous'] >= 40) & \
                                   (df_results['Age at diagnosis continuous'] < 50)]
    _51_to_60 = df_results[(df_results['Age at diagnosis continuous'] >= 50) & \
                                   (df_results['Age at diagnosis continuous'] < 60)]
    _61_to_70 = df_results[(df_results['Age at diagnosis continuous'] >= 60) & \
                                   (df_results['Age at diagnosis continuous'] < 70)]
    _71_to_80 = df_results[(df_results['Age at diagnosis continuous'] >= 70) & \
                                   (df_results['Age at diagnosis continuous'] < 80)]
    _81_to_90 = df_results[(df_results['Age at diagnosis continuous'] >= 80) & \
                                   (df_results['Age at diagnosis continuous'] < 90)]
    _90_and_above = df_results[df_results['Age at diagnosis continuous'] >= 90]
    
    group_list = [df_results, male, female, white, black, hispanic, asian, _30_and_below,\
                  _31_to_40, _41_to_50, _51_to_60, _61_to_70, _71_to_80,\
                 _81_to_90, _90_and_above]
    
    thresholds = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, \
              0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
    
    for t in thresholds:
        results_list = []
        for g in group_list:
            new_pred_y = (np.array(g['score y']) >= t).astype(np.int)
            
            results_list.append(matthews_corrcoef(np.array(g['true y']), new_pred_y))
    
        
        write_mcc_to_file(results_list, folder_name + '/' + valid_or_test + '_mcc_threshold_' + str(t) + '.csv')
        
    
    #generate_mcc_all_threshold_for_subgroups(folder_name, valid_or_test)
    #generate_all_threshold_for_subgroups(folder_name, 'test')
    
    # return
    # nothing need to be returned, results are written to files

In [5]:
def calculate_and_average_5_runs(cancer, folder_name):
    """
    average the performance of 5 runs of each experiment and calculate std
    folder_name: the folder containing the files for each of the 5 runs
    """
    valid_arr = []
    test_arr = []
    
    for i in range(1, 6):
        current_folder = folder_name + '/' + folder_name.split('/')[-1] + '-' + str(i)
        
        file = open(current_folder + '/best_epoch_number.txt', 'r')
        best_epo = file.read()
        file.close()
        #folder_name + '/' + valid_or_test + '_' + ss + '_all_thresholds_mcc.csv'
        current_folder += '/epo ' + best_epo
        
        # calculate mcc from valid and test results
        df_results_valid = pd.read_csv(current_folder + '/valid_results_data_frame_isotonic.csv')
        evaluate_all_subgroup_all_threshold(df_results_valid, cancer, current_folder, 'valid')
        df_results_test = pd.read_csv(current_folder + '/test_results_data_frame_isotonic.csv')
        evaluate_all_subgroup_all_threshold(df_results_test, cancer, current_folder, 'test')
        
        df_all = pd.read_csv(current_folder + '/valid_whole_group_threshold_summary.csv')
        #df_all = pd.read_csv(current_folder + '/epo ' + best_epo + '/test_whole_group_threshold_summary.csv')
        threshold = df_all['threshold'][0] # whole group threshold
        
        # calculate the mcc for selected models
        df = pd.read_csv(current_folder + '/valid_mcc_threshold_' + str(threshold) + '.csv')
        df_test = pd.read_csv(current_folder + '/test_mcc_threshold_' + str(threshold) + '.csv')
        
        df['threshold'] = threshold
        df_test['threshold'] = threshold
        
        df = df.drop(columns=['Unnamed: 0'])
        df_test = df_test.drop(columns=['Unnamed: 0'])
        
        valid_arr.append(df.values.flatten())
        test_arr.append(df_test.values.flatten())
        
    valid_mean = np.mean(valid_arr, axis=0)
    valid_std = np.std(valid_arr, axis=0)

    valid_mean = valid_mean.reshape(15, 2)
    valid_std = valid_std.reshape(15, 2)
    
    test_mean = np.mean(test_arr, axis=0)
    test_std = np.std(test_arr, axis=0)

    test_mean = test_mean.reshape(15, 2)
    test_std = test_std.reshape(15, 2)

    cols = ['MCC', 'threshold']
    rows = ['Whole', 'Gender_Male', 'Gender_Female', 'Ethnicity_White', 'Ethnicity_Black',\
             'Ethnicity_Hispanic', 'Ethnicity_Asian',\
             'Age<30', '30<=Age<40', '40<=Age<50',\
             '50<=Age<60', '60<=Age<70',\
             '70<=Age<80', '80<=Age<90', 'Age>=90']

    df_valid_mean = pd.DataFrame(data=valid_mean, index=rows, columns=cols)
    df_valid_std = pd.DataFrame(data=valid_std, index=rows, columns=cols)
    df_test_mean = pd.DataFrame(data=test_mean, index=rows, columns=cols)
    df_test_std = pd.DataFrame(data=test_std, index=rows, columns=cols)

    df_valid_mean.to_csv(folder_name + '/valid_whole_group_threshold_summary_mcc_mean.csv')
    df_valid_std.to_csv(folder_name + '/valid_whole_group_threshold_summary_mcc_std.csv')
    df_test_mean.to_csv(folder_name + '/test_whole_group_threshold_summary_mcc_mean.csv')
    df_test_std.to_csv(folder_name + '/test_whole_group_threshold_summary_mcc_std.csv')

In [6]:
def average_5_runs_subgroup(cancer, folder_name):
    """
    average the performance of 5 runs of each experiment and calculate std
    folder_name: the folder containing the files for each of the 5 runs
    """
    valid_arr = []
    test_arr = []
    
    for i in range(1, 6):
        current_folder = folder_name + '/' + folder_name.split('/')[-1] + '-' + str(i)
        
        file = open(current_folder + '/best_epoch_number.txt', 'r')
        best_epo = file.read()
        file.close()
        #folder_name + '/' + valid_or_test + '_' + ss + '_all_thresholds_mcc.csv'
        current_folder += '/epo ' + best_epo
        
        # calculate mcc from valid and test results
#         df_results_valid = pd.read_csv(current_folder + '/valid_results_data_frame_isotonic.csv')
#         evaluate_all_subgroup_all_threshold(df_results_valid, cancer, current_folder, 'valid')
#         df_results_test = pd.read_csv(current_folder + '/test_results_data_frame_isotonic.csv')
#         evaluate_all_subgroup_all_threshold(df_results_test, cancer, current_folder, 'test')
        
        df_all = pd.read_csv(current_folder + '/valid_subgroup_threshold_summary.csv')
        all_thresholds = df_all['threshold'].values
        #subgroups = df_all['Unnamed: 0'].values
        
        valid_val = []
        test_val = []
        
        for i in range(len(all_thresholds)):

            # calculate the mcc for selected models
            df = pd.read_csv(current_folder + '/valid_mcc_threshold_' + str(all_thresholds[i]) + '.csv')
            df_test = pd.read_csv(current_folder + '/test_mcc_threshold_' + str(all_thresholds[i]) + '.csv')

            valid_val.append([df['MCC'].values[i], all_thresholds[i]])
            test_val.append([df_test['MCC'].values[i], all_thresholds[i]])
        
        valid_arr.append(np.array(valid_val).flatten())
        test_arr.append(np.array(test_val).flatten())
        
    valid_mean = np.mean(valid_arr, axis=0)
    valid_std = np.std(valid_arr, axis=0)

    valid_mean = valid_mean.reshape(15, 2)
    valid_std = valid_std.reshape(15, 2)
    
    test_mean = np.mean(test_arr, axis=0)
    test_std = np.std(test_arr, axis=0)

    test_mean = test_mean.reshape(15, 2)
    test_std = test_std.reshape(15, 2)

    cols = ['MCC', 'threshold']
    rows = ['Whole', 'Gender_Male', 'Gender_Female', 'Ethnicity_White', 'Ethnicity_Black',\
             'Ethnicity_Hispanic', 'Ethnicity_Asian',\
             'Age<30', '30<=Age<40', '40<=Age<50',\
             '50<=Age<60', '60<=Age<70',\
             '70<=Age<80', '80<=Age<90', 'Age>=90']

    df_valid_mean = pd.DataFrame(data=valid_mean, index=rows, columns=cols)
    df_valid_std = pd.DataFrame(data=valid_std, index=rows, columns=cols)
    df_test_mean = pd.DataFrame(data=test_mean, index=rows, columns=cols)
    df_test_std = pd.DataFrame(data=test_std, index=rows, columns=cols)

    df_valid_mean.to_csv(folder_name + '/valid_subgroup_threshold_summary_mcc_mean.csv')
    df_valid_std.to_csv(folder_name + '/valid_subgroup_threshold_summary_mcc_std.csv')
    df_test_mean.to_csv(folder_name + '/test_subgroup_threshold_summary_mcc_mean.csv')
    df_test_std.to_csv(folder_name + '/test_subgroup_threshold_summary_mcc_std.csv')

In [7]:
def calculate_and_average_5_runs_dp(cancer, folder_name, original_folder_name, group_name):
    """
    average the performance of 5 runs of each experiment and calculate std
    folder_name: the folder containing the files for each of the 5 runs
    """
    valid_arr = []
    test_arr = []
    
    for i in range(1, 6):
        current_folder = folder_name + '/' + folder_name.split('/')[-1] + '-' + str(i)
        
        with open(current_folder + '/best_dp_unit_number_whole_group_threshold.txt', 'r') as file:
            best_unit = file.read().split(' ')[0] # number only
            
        if best_unit == '1':
            df = pd.read_csv(original_folder_name + '/valid_whole_group_threshold_summary_mcc_mean.csv')
            df_test = pd.read_csv(original_folder_name + '/test_whole_group_threshold_summary_mcc_mean.csv')
        
        else:
            current_folder += '/' + folder_name.split('/')[-1] + '-' + best_unit + 'u'
            
            with open(current_folder + '/best_epoch_number_whole_group_thre_' + group_name + '.txt', 'r') as file:
                best_epo = file.read()

            #folder_name + '/' + valid_or_test + '_' + ss + '_all_thresholds_mcc.csv'
            current_folder += '/epo ' + best_epo

            # calculate mcc from valid and test results
            df_results_valid = pd.read_csv(current_folder + '/valid_results_data_frame_isotonic.csv')
            evaluate_all_subgroup_all_threshold(df_results_valid, cancer, current_folder, 'valid')
            df_results_test = pd.read_csv(current_folder + '/test_results_data_frame_isotonic.csv')
            evaluate_all_subgroup_all_threshold(df_results_test, cancer, current_folder, 'test')

            df_all = pd.read_csv(current_folder + '/valid_whole_group_threshold_summary.csv')
            #df_all = pd.read_csv(current_folder + '/epo ' + best_epo + '/test_whole_group_threshold_summary.csv')
            threshold = df_all['threshold'][0] # whole group threshold

            # calculate the mcc for selected models
            df = pd.read_csv(current_folder + '/valid_mcc_threshold_' + str(threshold) + '.csv')
            df_test = pd.read_csv(current_folder + '/test_mcc_threshold_' + str(threshold) + '.csv')

            df['threshold'] = threshold
            df_test['threshold'] = threshold
        
        df = df.drop(columns=['Unnamed: 0'])
        df_test = df_test.drop(columns=['Unnamed: 0'])
        
        valid_arr.append(df.values.flatten())
        test_arr.append(df_test.values.flatten())
        
    valid_mean = np.mean(valid_arr, axis=0)
    valid_std = np.std(valid_arr, axis=0)

    valid_mean = valid_mean.reshape(15, 2)
    valid_std = valid_std.reshape(15, 2)
    
    test_mean = np.mean(test_arr, axis=0)
    test_std = np.std(test_arr, axis=0)

    test_mean = test_mean.reshape(15, 2)
    test_std = test_std.reshape(15, 2)

    cols = ['MCC', 'threshold']
    rows = ['Whole', 'Gender_Male', 'Gender_Female', 'Ethnicity_White', 'Ethnicity_Black',\
             'Ethnicity_Hispanic', 'Ethnicity_Asian',\
             'Age<30', '30<=Age<40', '40<=Age<50',\
             '50<=Age<60', '60<=Age<70',\
             '70<=Age<80', '80<=Age<90', 'Age>=90']

    df_valid_mean = pd.DataFrame(data=valid_mean, index=rows, columns=cols)
    df_valid_std = pd.DataFrame(data=valid_std, index=rows, columns=cols)
    df_test_mean = pd.DataFrame(data=test_mean, index=rows, columns=cols)
    df_test_std = pd.DataFrame(data=test_std, index=rows, columns=cols)

    df_valid_mean.to_csv(folder_name + '/valid_whole_group_threshold_summary_mcc_mean.csv')
    df_valid_std.to_csv(folder_name + '/valid_whole_group_threshold_summary_mcc_std.csv')
    df_test_mean.to_csv(folder_name + '/test_whole_group_threshold_summary_mcc_mean.csv')
    df_test_std.to_csv(folder_name + '/test_whole_group_threshold_summary_mcc_std.csv')

In [7]:
def output_5_runs(folder_name):
    """
    put results of each individual runs into 1 file
    folder_name: the folder containing the files for each of the 5 runs
    """
    valid_arr = []
    test_arr = []
    
    for i in range(1, 6):
        current_folder = folder_name + '/' + folder_name.split('/')[-1] + '-' + str(i)
        
        file = open(current_folder + '/best_epoch_number.txt', 'r')
        best_epo = file.read()
        file.close()
        
        df_all = pd.read_csv(current_folder + '/epo ' + best_epo + '/valid_whole_group_threshold_summary.csv')
        threshold = df_all['threshold'][0] # whole group threshold
        
        with open(current_folder + '/epo ' + best_epo + '/valid_mcc_threshold_' + str(threshold) + '.csv', 'r') as v:
            valid_arr.append(v.read())
        with open(current_folder + '/epo ' + best_epo + '/test_mcc_threshold_' + str(threshold) + '.csv', 'r') as t:
            test_arr.append(t.read())
    
    
    with open(folder_name + '/valid_mcc_5_run_results.csv', 'w') as vout:
        vout.write('\n\n'.join(valid_arr))
        
    with open(folder_name + '/test_mcc_5_run_results.csv', 'w') as tout:
        tout.write('\n\n'.join(test_arr))

In [25]:
def output_5_runs_dp(dp_folder_name, original_folder_name, group_name):
    """
    put results of each individual runs into 1 file
    folder_name: the folder containing the files for each of the 5 runs
    """
    valid_arr = []
    test_arr = []
    
    for i in range(1, 6):
        folder_each_run = dp_folder_name + '/' + dp_folder_name.split('/')[-1] + '-' + str(i)
        
        # open best unit
        unit_file = open(folder_each_run + '/best_dp_unit_number_whole_group_threshold.txt', 'r')
        best_unit = unit_file.read().split(' ')[0] # number only
        unit_file.close()
        
         # open best epoch
        if best_unit == '1':
            with open(original_folder_name + '/valid_whole_group_threshold_summary_mcc_mean.csv', 'r') as v:
                valid_arr.append(v.read())
            with open(original_folder_name + '/test_whole_group_threshold_summary_mcc_mean.csv', 'r') as t:
                test_arr.append(t.read())
                
        else:
            best_unit_folder = folder_each_run + '/' + dp_folder_name.split('/')[-1] + '-' + best_unit + 'u'
            epoch_file = open(best_unit_folder + '/best_epoch_number_whole_group_thre_' + group_name + '.txt', 'r')
            best_epo = epoch_file.read()
            epoch_file.close()
            
            df_all = pd.read_csv(best_unit_folder + '/epo ' + best_epo + '/valid_whole_group_threshold_summary.csv')
            threshold = df_all['threshold'][0] # whole group threshold
            #print(threshold)
        
            # read summary files
            with open(best_unit_folder + '/epo ' + best_epo + '/valid_mcc_threshold_' + str(threshold) + '.csv') as v:
                valid_arr.append(v.read())
            with open(best_unit_folder + '/epo ' + best_epo + '/test_mcc_threshold_' + str(threshold) + '.csv') as t:
                test_arr.append(t.read())

        
    with open(dp_folder_name + '/valid_mcc_5_run_results.csv', 'w') as vout:
        vout.write('\n\n'.join(valid_arr))
        
    with open(dp_folder_name + '/test_mcc_5_run_results.csv', 'w') as tout:
        tout.write('\n\n'.join(test_arr))

In [ ]:
###################################################
## test and run
###################################################

In [8]:
output_5_runs('../experiments/bcs-original')

In [10]:
output_5_runs('../experiments/bcs-adasyn')
output_5_runs('../experiments/bcs-gamma')
output_5_runs('../experiments/bcs-over')
output_5_runs('../experiments/bcs-nearmiss1')
output_5_runs('../experiments/bcs-nearmiss3')
output_5_runs('../experiments/bcs-distant')
output_5_runs('../experiments/bcs-smote')
output_5_runs('../experiments/bcs-under')

In [27]:
output_5_runs('../experiments/bcs-reweight')

In [23]:
output_5_runs_dp('../experiments/bcs-dp-asian', '../experiments/bcs-original', 'Ethnicity_Asian')
output_5_runs_dp('../experiments/bcs-dp-4050', '../experiments/bcs-original', '40<=Age<50')

0.25
0.3
0.25
0.25
0.25
0.25
0.25
0.25
0.25
0.25


In [26]:
output_5_runs_dp('../experiments/bcs-dpreweight-asian', '../experiments/bcs-original', 'Ethnicity_Asian')
output_5_runs_dp('../experiments/bcs-dpreweight-4050', '../experiments/bcs-original', '40<=Age<50')

In [28]:
output_5_runs('../experiments/lcs-original')
output_5_runs('../experiments/lcs-adasyn')
output_5_runs('../experiments/lcs-gamma')
output_5_runs('../experiments/lcs-over')
output_5_runs('../experiments/lcs-nearmiss1')
output_5_runs('../experiments/lcs-nearmiss3')
output_5_runs('../experiments/lcs-distant')
output_5_runs('../experiments/lcs-smote')
output_5_runs('../experiments/lcs-under')
output_5_runs('../experiments/lcs-reweight')

In [30]:
output_5_runs_dp('../experiments/lcs-dp-black', '../experiments/lcs-original', 'Ethnicity_Black')
output_5_runs_dp('../experiments/lcs-dp-8090', '../experiments/lcs-original', '80<=Age<90')

In [61]:
average_5_runs_subgroup('breast', '../experiments/bcs-original')

In [62]:
average_5_runs_subgroup('lung', '../experiments/lcs-original')

In [ ]:
calculate_and_average_5_runs('breast', '../experiments/bcs-original')

In [ ]:
calculate_and_average_5_runs('breast', '../experiments/bcs-under')
calculate_and_average_5_runs('breast', '../experiments/bcs-nearmiss1')
calculate_and_average_5_runs('breast', '../experiments/bcs-nearmiss3')
calculate_and_average_5_runs('breast', '../experiments/bcs-distant')
calculate_and_average_5_runs('breast', '../experiments/bcs-over')
calculate_and_average_5_runs('breast', '../experiments/bcs-smote')
calculate_and_average_5_runs('breast', '../experiments/bcs-adasyn')
calculate_and_average_5_runs('breast', '../experiments/bcs-gamma')
calculate_and_average_5_runs('breast', '../experiments/bcs-reweight')

In [ ]:
calculate_and_average_5_runs('lung', '../experiments/lcs-original')
calculate_and_average_5_runs('lung', '../experiments/lcs-under')
calculate_and_average_5_runs('lung', '../experiments/lcs-nearmiss1')
calculate_and_average_5_runs('lung', '../experiments/lcs-nearmiss3')
calculate_and_average_5_runs('lung', '../experiments/lcs-distant')
calculate_and_average_5_runs('lung', '../experiments/lcs-over')
calculate_and_average_5_runs('lung', '../experiments/lcs-smote')
calculate_and_average_5_runs('lung', '../experiments/lcs-adasyn')
calculate_and_average_5_runs('lung', '../experiments/lcs-gamma')
calculate_and_average_5_runs('lung', '../experiments/lcs-reweight')

In [ ]:
calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dp-asian', '../experiments/bcs-original', 'Ethnicity_Asian')

In [ ]:
calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dp-black', '../experiments/bcs-original', 'Ethnicity_Black')
calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dp-hispanic', '../experiments/bcs-original', 'Ethnicity_Hispanic')
#calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dp-3040', '../experiments/bcs-original', '30<=Age<40')
calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dp-4050', '../experiments/bcs-original', '40<=Age<50')
calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dp-age90', '../experiments/bcs-original', 'Age>=90')
calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dp-age30', '../experiments/bcs-original', 'Age<30')

In [ ]:
calculate_and_average_5_runs_dp('lung', '../experiments/lcs-dp-asian', '../experiments/lcs-original', 'Ethnicity_Asian')
calculate_and_average_5_runs_dp('lung', '../experiments/lcs-dp-black', '../experiments/lcs-original', 'Ethnicity_Black')
calculate_and_average_5_runs_dp('lung', '../experiments/lcs-dp-hispanic', '../experiments/lcs-original', 'Ethnicity_Hispanic')
calculate_and_average_5_runs_dp('lung', '../experiments/lcs-dp-3040', '../experiments/lcs-original', '30<=Age<40')
calculate_and_average_5_runs_dp('lung', '../experiments/lcs-dp-age90', '../experiments/lcs-original', 'Age>=90')
calculate_and_average_5_runs_dp('lung', '../experiments/lcs-dp-8090', '../experiments/lcs-original', '80<=Age<90')

In [12]:
calculate_and_average_5_runs('breast', '../experiments/bcs-original-different-parameters/bcs-original-5-30-0.2')
calculate_and_average_5_runs('breast', '../experiments/bcs-original-different-parameters/bcs-original-10-50-0.3')

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [13]:
calculate_and_average_5_runs('lung', '../experiments/lcs-original-different-parameters/lcs-original-5-30-0.2')
calculate_and_average_5_runs('lung', '../experiments/lcs-original-different-parameters/lcs-original-10-50-0.3')

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

...

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [14]:
calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dpreweight-asian', '../experiments/bcs-original', 'Ethnicity_Asian')
calculate_and_average_5_runs_dp('breast', '../experiments/bcs-dpreweight-4050', '../experiments/bcs-original', '40<=Age<50')

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/

/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/lib/python3.9/site-packages/sklearn/metrics/_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
/home/wenjia/.local/